## Notes
I have yet to conclude training the model, but I was to excited to share it finally working after a long time.
On a side note, however, it does seem to overfit before the end of even the first epoch, but we need to conclude a whole training session before jumping to any conclusions

# Word Prediction using TDNN implemented in tf.keras
This notebook is an attempt at implementing a Time Delay Neural Network for word prediction in the ptb dataset

In [3]:
# Imports
from __future__ import print_function
import tensorboard as tf
from tensorflow import keras
import reader
import numpy as np

In [66]:
# Constants
window_size = 20 # defines the past lookup for determining the following word
path = "data/simple-examples/data"

The following cell obtain the data using the reader.py file

In [73]:
train_data, valid_data, test_data, vocab_size, word_to_id = reader.ptb_raw_data(path)
x_train = train_data[:-1]
x_train = [np.asarray(x_train[i:i+window_size]) for i in range(len(x_train)-window_size)]
x_train = np.asarray(x_train)
y_train = np.asarray(train_data[1:-window_size])
#y_train = keras.preprocessing.text.one_hot(y_train, vocab_size)
x_valid = valid_data[:-1]
x_valid = [np.asarray(x_valid[i:i+window_size]) for i in range(len(x_valid)-window_size)]
x_valid = np.asarray(x_valid)
y_valid = valid_data[1:-window_size]
y_valid = np.asarray(y_valid)
x_test = test_data[:-1]
x_test = [np.asarray(x_test[i:i+window_size]) for i in range(len(x_test)-window_size)]
x_test = np.asarray(x_test)
y_test = test_data[1:-window_size]
y_test = np.asarray(y_test)
id_to_word = {value: key for (key, value) in word_to_id.items()}

Following next, we have an auxiliary function which decodes the ids and give us the original sentences

In [10]:
def decode_text(text):
    return ' '.join([id_to_word.get(i, '?') for i in text])

In [22]:
decode_text (x_train[40])

'<eos> mr. <unk> is chairman of <unk> n.v. the dutch publishing group <eos> rudolph <unk> N years old and former'

In [54]:
print(x_train.shape)
print(y_train.shape)

(929568, 20)
(929568,)


As we can see, each index of the input has a length of 20 words, as defined in _window-length_
<br>
With our data already processed, we can finally create our model.

In [67]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_length = window_size))
#model.add(keras.layers.Flatten())
print(model.output_shape)
model.add(keras.layers.Conv1D(filters = 1, kernel_size = 3, padding = "same", activation = keras.activations.tanh))
print(model.output_shape)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv1D(filters = 1, kernel_size = 3, padding = "same", activation = keras.activations.tanh))
print(model.output_shape)
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
print(model.output_shape)
model.add(keras.layers.Dense(vocab_size, activation = keras.activations.softmax))
print(model.output_shape)

(None, 20, 16)
(None, 20, 1)
(None, 20, 1)
(None, 20)
(None, 10000)


In [68]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 20, 1)             49        
_________________________________________________________________
dropout_15 (Dropout)         (None, 20, 1)             0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 20, 1)             4         
_________________________________________________________________
dropout_16 (Dropout)         (None, 20, 1)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10000)             210000    
Total para

In [69]:
model.compile(
    loss = keras.losses.sparse_categorical_crossentropy,
    optimizer = keras.optimizers.Adadelta(),
    metrics = [keras.metrics.categorical_accuracy] # remember to later change to sparse_categorical_accuracy (this is the cause for strange eval)
)

In [70]:
model.fit(x_train, y_train,
          epochs = 12,
          verbose = 1,
          validation_data = (x_valid, y_valid),
          shuffle = False
)

Train on 929568 samples, validate on 73739 samples
Epoch 1/12
929568/929568 [==============================] - 900s 968us/step - loss: 6.8354 - categorical_accuracy: 0.5550 - val_loss: 6.7577 - val_categorical_accuracy: 0.0000e+00
Epoch 2/12
929568/929568 [==============================] - 814s 876us/step - loss: 6.7952 - categorical_accuracy: 0.5676 - val_loss: 6.7580 - val_categorical_accuracy: 1.0000
Epoch 3/12
929568/929568 [==============================] - 759s 817us/step - loss: 6.7873 - categorical_accuracy: 0.5706 - val_loss: 6.7579 - val_categorical_accuracy: 1.0000
Epoch 4/12
929568/929568 [==============================] - 749s 805us/step - loss: 6.7811 - categorical_accuracy: 0.5747 - val_loss: 6.7594 - val_categorical_accuracy: 0.0000e+00
Epoch 5/12
929568/929568 [==============================] - 939s 1ms/step - loss: 6.7768 - categorical_accuracy: 0.5700 - val_loss: 6.7599 - val_categorical_accuracy: 0.0000e+00
Epoch 6/12
929568/929568 [==============================] -

In [74]:
# I expect to be able to run this someday
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 5.541705676066773
Test accuracy: 0.18199468504653618
